In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import nltk
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
/kaggle/input/amazon-ml-challenge-2021-hackerearth/sample_submission.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/train.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/test.csv


In [2]:
train_data = pd.read_csv('../input/amazon-ml-challenge-2021-hackerearth/train.csv',sep=',', header=None,escapechar="\\",quoting=csv.QUOTE_NONE,error_bad_lines=False)
print(train_data.shape)

test_data = pd.read_csv('../input/amazon-ml-challenge-2021-hackerearth/test.csv',sep=',', header=None,escapechar="\\",quoting=csv.QUOTE_NONE,error_bad_lines=False)
print(test_data.shape)


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2903025, 5)
(110776, 5)


In [3]:
pd.set_option('display.width', 500)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", -1)
train_data = train_data.iloc[1:80000 , :]
train = train_data.iloc[0:50000,:]
test = train_data.iloc[50001:80000,:]
test_data = test
train_data = train

# print(test_data.iloc[0:10,1])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
train_data.rename(columns={0: 'TITLE',1:"DESCRIPTION",2:"BULLET_POINTS",3:"BRAND",4:"BROWSE_NODE_ID"},inplace = True)
test_data.rename(columns={0: 'TITLE',1:"DESCRIPTION",2:"BULLET_POINTS",3:"BRAND",4:"BROWSE_NODE_ID"},inplace = True)
for col in train_data.columns:
    print(col)
for col in test_data.columns:
    print(col)    

TITLE
DESCRIPTION
BULLET_POINTS
BRAND
BROWSE_NODE_ID
TITLE
DESCRIPTION
BULLET_POINTS
BRAND
BROWSE_NODE_ID


In [5]:
train_data.fillna("empty",inplace = True)
test_data.fillna("empty",inplace = True)

In [6]:
# test_data = test_data.iloc[0:50000,:]
# train_data = train_data.iloc[0:50000,:]

In [7]:
X_train = train_data.iloc[:,0:4]
Y_train = train_data.iloc[:,4]
Test_X = test_data.iloc[:,0:4]
Test_Y = test_data.iloc[:,4]

In [8]:
X_train = X_train['TITLE']
Test_X = Test_X['TITLE']
print(X_train.iloc[0:10])
print(Test_X.iloc[0:10])

1     Pete The Cat Bedtime Blues Doll, 14.5 Inch                                                                                              
2     The New Yorker NYHM014 Refrigerator Magnet, 2 x 3.5                                                                                     
3     The Ultimate Self-Sufficiency Handbook: A Complete Guide to Baking, Crafts, Gardening, Preserving Your Harvest, Raising Animals and More
4     Amway Nutrilite Kids Chewable Iron Tablets (100)                                                                                        
5     Teacher Planner Company A4 6 Lesson Academic Teacher Planner - Leatherette Pink                                                         
6     Men'S Full Sleeve Raglan T-Shirts Denim T-Shirt M Size Only                                                                             
7     Glance Women's Wallet (Black) (LW-21)                                                                                                   

In [9]:
print(X_train.shape)
print(Y_train.shape)
print(Test_X.shape)
print(Test_Y.shape)

(50000,)
(50000,)
(29998,)
(29998,)


In [10]:
# X_train['TITLE']= X_train['TITLE'].apply(lambda x: x.lower())
# X_train['DESCRIPTION']= X_train['DESCRIPTION'].apply(lambda x: x.lower())
# X_train['BULLET_POINTS']= X_train['BULLET_POINTS'].apply(lambda x: x.lower())
# X_train['BRAND']= X_train['BRAND'].apply(lambda x: x.lower())


# Test_X['TITLE']= Test_X['TITLE'].apply(lambda x: x.lower())
# Test_X['DESCRIPTION']= Test_X['DESCRIPTION'].apply(lambda x: x.lower())
# Test_X['BULLET_POINTS']= Test_X['BULLET_POINTS'].apply(lambda x: x.lower())
# Test_X['BRAND']= Test_X['BRAND'].apply(lambda x: x.lower())

In [11]:
# cvec = CountVectorizer()
# # df_train = pd.DataFrame(cvec.transform(X_train).todense(),columns = cvec.get_feature_names())
# # df_test = pd.DataFrame(cvec.transform(Test_X).todense(),columns = cvec.get_feature_names())
# # print(df_train.shape)
# # print(df_test.shape)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X_train)
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)


In [12]:

# model = make_pipeline(TfidfVectorizer(), MultinomialNB())
# model.fit(cvec.fit_transform(X_train), Y_train)
# predicted_categories = model.predict(df_test['TITLE'])

# print("The accuracy is {}".format(accuracy_score(Test_Y['TITLE'], predicted_categories)))


Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Y_train)

predictions_NB = Naive.predict(Test_X_Tfidf)

print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  32.138809253950264
